In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA, FastICA, KernelPCA, TruncatedSVD
from sklearn.manifold import TSNE, Isomap, LocallyLinearEmbedding
from sklearn.random_projection import GaussianRandomProjection
from sklearn.metrics import mean_squared_error
from umap import UMAP
from sklearn.linear_model import Ridge, ElasticNet, SGDRegressor, BayesianRidge, LinearRegression, RANSACRegressor, TheilSenRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge

# Load the data
file_path = "data_all_numerical_select_reduced.xlsx"
data = pd.read_excel(file_path)

# Define columns
data_columns = [
    'OF2', 'OF3', 'OF4', 'OF5', 'OF6', 'OF7', 'OF8', 'OF9', 'OF10', 'OF11', 'OF12', 'OF13', 'OF14', 'OF15', 'OF16', 'OF17', 'OF18', 'OF19', 'OF20',
    'OF21', 'OF22', 'OF23', 'OF24', 'OF25', 'OF26', 'OF27', 'OF28', 'OF29', 'OF30', 'OF31', 'OF32', 'OF33', 'OF34', 'OF37', 'OF38', 'F1', 'F2',
    'F3_1', 'F3_2', 'F3_3', 'F3_4', 'F3_5', 'F3_6', 'F3_7', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16',
    'F17', 'F18', 'F19', 'F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28', 'F29', 'F30', 'F31', 'F32', 'F33', 'F34', 'F35', 'F36',
    'F37', 'F38', 'F39', 'F40', 'F41', 'F42', 'F43', 'F44', 'F45', 'F46', 'F47', 'F48', 'F49', 'F50', 'F51', 'F52', 'F53', 'F54', 'F55', 'F56',
    'F57', 'F58', 'F59', 'F62', 'F63', 'F64', 'F65', 'F66', 'F67', 'F68', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'PC', 'FC', 'WRI', 'SVT', 'VCHWC',
    'HWCC', 'MC', 'PP', 'ST', 'SWP', 'DP', 'ADLM', 'ATDO', 'AOD'
]

results_columns = ['WS']

# Prepare data for regression
X = data[data_columns]
y = data[results_columns[0]]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Implement dimensionality reduction techniques
dimensionality_reduction_techniques = {
    "PCA": PCA(n_components=10),
    "t-SNE": TSNE(n_components=2),
    "UMAP": UMAP(n_components=10),
    "Isomap": Isomap(n_components=10),
    "LLE": LocallyLinearEmbedding(n_components=10),
    "Autoencoders": TruncatedSVD(n_components=10),  # Assuming TruncatedSVD as a simple autoencoder
    "ICA": FastICA(n_components=10),
    "Kernel PCA": KernelPCA(n_components=10),
    "Random Projection": GaussianRandomProjection(n_components=10)  # Assuming GaussianRandomProjection
}

# Define the models
models = [
    Ridge(), DecisionTreeRegressor(), GradientBoostingRegressor(),
    RandomForestRegressor(), AdaBoostRegressor(), KNeighborsRegressor(),
    MLPRegressor(max_iter=200), ElasticNet(max_iter=1000), SGDRegressor(max_iter=1000),
    SVR(cache_size=1000), BayesianRidge(max_iter=1000), KernelRidge(),
    LinearRegression(), RANSACRegressor(), TheilSenRegressor()
]

# Train and evaluate each model with each dimensionality reduction technique
results = {}
predictions = {}
for name, reducer in dimensionality_reduction_techniques.items():
    results[name] = {}
    predictions[name] = {}
    for model in models:
        # Transform the data
        if name == "LDA":
            X_train_reduced = reducer.fit_transform(X_train_scaled, y_train)
            X_test_reduced = reducer.transform(X_test_scaled)
        elif name == "t-SNE":
            X_concatenated = np.concatenate((X_train_scaled, X_test_scaled), axis=0)
            X_reduced = reducer.fit_transform(X_concatenated)
            X_train_reduced = X_reduced[:len(X_train_scaled)]
            X_test_reduced = X_reduced[len(X_train_scaled):]
        else:
            X_train_reduced = reducer.fit_transform(X_train_scaled)
            X_test_reduced = reducer.transform(X_test_scaled)
        
        # Train the model
        model.fit(X_train_reduced, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test_reduced)
        
        # Calculate MSE
        mse = mean_squared_error(y_test, y_pred)
        
        # Store results
        results[name][model.__class__.__name__] = mse
        predictions[name][model.__class__.__name__] = y_pred

# Print results
for name, model_results in results.items():
    print(f"Results using {name}:")
    for model, mse in model_results.items():
        print(f"MSE using {model}: {mse}")

# Show 5 predictions from each reduction algorithm along with the expected value
for name, preds in predictions.items():
    print(f"Predictions using {name}:")
    for model, y_pred in preds.items():
        print(f"Model: {model}")
        for i in range(5):
            print(f"Example {i+1}: Predicted: {y_pred[i]}, Expected: {y_test.values[i]}")


C:\Users\marcb\anaconda3\envs\greystone\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\marcb\anaconda3\envs\greystone\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\marcb\anaconda3\envs\greystone\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\marcb\anaconda3\envs\greystone\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\mar

Results using PCA:
MSE using Ridge: 345.0922255687727
MSE using DecisionTreeRegressor: 6.248342713290851
MSE using GradientBoostingRegressor: 2.6881884621869667
MSE using RandomForestRegressor: 2.3883333059392675
MSE using AdaBoostRegressor: 4.0572679844119435
MSE using KNeighborsRegressor: 2.298464921792962
MSE using MLPRegressor: 27819.825832240316
MSE using ElasticNet: 1364.3878714671157
MSE using SGDRegressor: 595.3544419648862
MSE using SVR: 2.341463426373836
MSE using BayesianRidge: 512.7712869978558
MSE using KernelRidge: 337.9334208214867
MSE using LinearRegression: 342.3392684276132
MSE using RANSACRegressor: 4888.84099609512
MSE using TheilSenRegressor: 471.567516366982
Results using t-SNE:
MSE using Ridge: 4.18614382963218
MSE using DecisionTreeRegressor: 4.179383994429173
MSE using GradientBoostingRegressor: 6.227245131892635
MSE using RandomForestRegressor: 2.7515517060064703
MSE using AdaBoostRegressor: 4.023135600229982
MSE using KNeighborsRegressor: 3.0937683143639263
M

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA, FastICA, KernelPCA, TruncatedSVD
from sklearn.manifold import TSNE, Isomap, LocallyLinearEmbedding
from sklearn.random_projection import GaussianRandomProjection
from sklearn.metrics import mean_squared_error
from umap import UMAP
from sklearn.linear_model import Ridge, ElasticNet, SGDRegressor, BayesianRidge, LinearRegression, RANSACRegressor, TheilSenRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge

# Load the data
file_path = "data_all_numerical_select_reduced.xlsx"
data = pd.read_excel(file_path)

# Define columns
data_columns = [
    'OF2', 'OF3', 'OF4', 'OF5', 'OF6', 'OF7', 'OF8', 'OF9', 'OF10', 'OF11', 'OF12', 'OF13', 'OF14', 'OF15', 'OF16', 'OF17', 'OF18', 'OF19', 'OF20',
    'OF21', 'OF22', 'OF23', 'OF24', 'OF25', 'OF26', 'OF27', 'OF28', 'OF29', 'OF30', 'OF31', 'OF32', 'OF33', 'OF34', 'OF37', 'OF38', 'F1', 'F2',
    'F3_1', 'F3_2', 'F3_3', 'F3_4', 'F3_5', 'F3_6', 'F3_7', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16',
    'F17', 'F18', 'F19', 'F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28', 'F29', 'F30', 'F31', 'F32', 'F33', 'F34', 'F35', 'F36',
    'F37', 'F38', 'F39', 'F40', 'F41', 'F42', 'F43', 'F44', 'F45', 'F46', 'F47', 'F48', 'F49', 'F50', 'F51', 'F52', 'F53', 'F54', 'F55', 'F56',
    'F57', 'F58', 'F59', 'F62', 'F63', 'F64', 'F65', 'F66', 'F67', 'F68', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'PC', 'FC', 'WRI', 'SVT', 'VCHWC',
    'HWCC', 'MC', 'PP', 'ST', 'SWP', 'DP', 'ADLM', 'ATDO', 'AOD'
]

results_columns = ['NR']

# Prepare data for regression
X = data[data_columns]
y = data[results_columns[0]]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Implement dimensionality reduction techniques
dimensionality_reduction_techniques = {
    "PCA": PCA(n_components=10),
    "t-SNE": TSNE(n_components=2),
    "UMAP": UMAP(n_components=10),
    "Isomap": Isomap(n_components=10),
    "LLE": LocallyLinearEmbedding(n_components=10),
    "Autoencoders": TruncatedSVD(n_components=10),  # Assuming TruncatedSVD as a simple autoencoder
    "ICA": FastICA(n_components=10),
    "Kernel PCA": KernelPCA(n_components=10),
    "Random Projection": GaussianRandomProjection(n_components=10)  # Assuming GaussianRandomProjection
}

# Define the models
models = [
    Ridge(), DecisionTreeRegressor(), GradientBoostingRegressor(),
    RandomForestRegressor(), AdaBoostRegressor(), KNeighborsRegressor(),
    MLPRegressor(max_iter=200), ElasticNet(max_iter=1000), SGDRegressor(max_iter=1000),
    SVR(cache_size=1000), BayesianRidge(max_iter=1000), KernelRidge(),
    LinearRegression(), RANSACRegressor(), TheilSenRegressor()
]

# Train and evaluate each model with each dimensionality reduction technique
results = {}
predictions = {}
for name, reducer in dimensionality_reduction_techniques.items():
    results[name] = {}
    predictions[name] = {}
    for model in models:
        # Transform the data
        if name == "LDA":
            X_train_reduced = reducer.fit_transform(X_train_scaled, y_train)
            X_test_reduced = reducer.transform(X_test_scaled)
        elif name == "t-SNE":
            X_concatenated = np.concatenate((X_train_scaled, X_test_scaled), axis=0)
            X_reduced = reducer.fit_transform(X_concatenated)
            X_train_reduced = X_reduced[:len(X_train_scaled)]
            X_test_reduced = X_reduced[len(X_train_scaled):]
        else:
            X_train_reduced = reducer.fit_transform(X_train_scaled)
            X_test_reduced = reducer.transform(X_test_scaled)
        
        # Train the model
        model.fit(X_train_reduced, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test_reduced)
        
        # Calculate MSE
        mse = mean_squared_error(y_test, y_pred)
        
        # Store results
        results[name][model.__class__.__name__] = mse
        predictions[name][model.__class__.__name__] = y_pred

# Print results
for name, model_results in results.items():
    print(f"Results using {name}:")
    for model, mse in model_results.items():
        print(f"MSE using {model}: {mse}")

# Show 5 predictions from each reduction algorithm along with the expected value
for name, preds in predictions.items():
    print(f"Predictions using {name}:")
    for model, y_pred in preds.items():
        print(f"Model: {model}")
        for i in range(5):
            print(f"Example {i+1}: Predicted: {y_pred[i]}, Expected: {y_test.values[i]}")


C:\Users\marcb\anaconda3\envs\greystone\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\marcb\anaconda3\envs\greystone\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\marcb\anaconda3\envs\greystone\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\marcb\anaconda3\envs\greystone\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\mar

Results using PCA:
MSE using Ridge: 2024.4529527759933
MSE using DecisionTreeRegressor: 4.84583815698709
MSE using GradientBoostingRegressor: 5.3206822907368245
MSE using RandomForestRegressor: 2.711291716469124
MSE using AdaBoostRegressor: 6.090933449221054
MSE using KNeighborsRegressor: 3.0142751152248444
MSE using MLPRegressor: 45535.01530770608
MSE using ElasticNet: 1905.2041763097473
MSE using SGDRegressor: 2670.381862645266
MSE using SVR: 2.639051858168193
MSE using BayesianRidge: 2264.5085370631723
MSE using KernelRidge: 1982.544158067012
MSE using LinearRegression: 2019.5542424487114
MSE using RANSACRegressor: 833.8876924783393
MSE using TheilSenRegressor: 2105.138893032411
Results using t-SNE:
MSE using Ridge: 5.114694066849844
MSE using DecisionTreeRegressor: 8.430307885199154
MSE using GradientBoostingRegressor: 3.4474193171183884
MSE using RandomForestRegressor: 4.218974433889901
MSE using AdaBoostRegressor: 4.358222165952156
MSE using KNeighborsRegressor: 3.610163019651660

In [4]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA, FastICA, KernelPCA, TruncatedSVD
from sklearn.manifold import TSNE, Isomap, LocallyLinearEmbedding
from sklearn.random_projection import GaussianRandomProjection
from sklearn.metrics import mean_squared_error
from umap import UMAP
from sklearn.linear_model import Ridge, ElasticNet, SGDRegressor, BayesianRidge, LinearRegression, RANSACRegressor, TheilSenRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge

# Load the data
file_path = "data_all_numerical_select_reduced.xlsx"
data = pd.read_excel(file_path)

# Define columns
data_columns = [
    'OF2', 'OF3', 'OF4', 'OF5', 'OF6', 'OF7', 'OF8', 'OF9', 'OF10', 'OF11', 'OF12', 'OF13', 'OF14', 'OF15', 'OF16', 'OF17', 'OF18', 'OF19', 'OF20',
    'OF21', 'OF22', 'OF23', 'OF24', 'OF25', 'OF26', 'OF27', 'OF28', 'OF29', 'OF30', 'OF31', 'OF32', 'OF33', 'OF34', 'OF37', 'OF38', 'F1', 'F2',
    'F3_1', 'F3_2', 'F3_3', 'F3_4', 'F3_5', 'F3_6', 'F3_7', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16',
    'F17', 'F18', 'F19', 'F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28', 'F29', 'F30', 'F31', 'F32', 'F33', 'F34', 'F35', 'F36',
    'F37', 'F38', 'F39', 'F40', 'F41', 'F42', 'F43', 'F44', 'F45', 'F46', 'F47', 'F48', 'F49', 'F50', 'F51', 'F52', 'F53', 'F54', 'F55', 'F56',
    'F57', 'F58', 'F59', 'F62', 'F63', 'F64', 'F65', 'F66', 'F67', 'F68', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'PC', 'FC', 'WRI', 'SVT', 'VCHWC',
    'HWCC', 'MC', 'PP', 'ST', 'SWP', 'DP', 'ADLM', 'ATDO', 'AOD'
]

results_columns = ['PR']

# Prepare data for regression
X = data[data_columns]
y = data[results_columns[0]]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Implement dimensionality reduction techniques
dimensionality_reduction_techniques = {
    "PCA": PCA(n_components=10),
    "t-SNE": TSNE(n_components=2),
    "UMAP": UMAP(n_components=10),
    "Isomap": Isomap(n_components=10),
    "LLE": LocallyLinearEmbedding(n_components=10),
    "Autoencoders": TruncatedSVD(n_components=10),  # Assuming TruncatedSVD as a simple autoencoder
    "ICA": FastICA(n_components=10),
    "Kernel PCA": KernelPCA(n_components=10),
    "Random Projection": GaussianRandomProjection(n_components=10)  # Assuming GaussianRandomProjection
}

# Define the models
models = [
    Ridge(), DecisionTreeRegressor(), GradientBoostingRegressor(),
    RandomForestRegressor(), AdaBoostRegressor(), KNeighborsRegressor(),
    MLPRegressor(max_iter=200), ElasticNet(max_iter=1000), SGDRegressor(max_iter=1000),
    SVR(cache_size=1000), BayesianRidge(max_iter=1000), KernelRidge(),
    LinearRegression(), RANSACRegressor(), TheilSenRegressor()
]

# Train and evaluate each model with each dimensionality reduction technique
results = {}
predictions = {}
for name, reducer in dimensionality_reduction_techniques.items():
    results[name] = {}
    predictions[name] = {}
    for model in models:
        # Transform the data
        if name == "LDA":
            X_train_reduced = reducer.fit_transform(X_train_scaled, y_train)
            X_test_reduced = reducer.transform(X_test_scaled)
        elif name == "t-SNE":
            X_concatenated = np.concatenate((X_train_scaled, X_test_scaled), axis=0)
            X_reduced = reducer.fit_transform(X_concatenated)
            X_train_reduced = X_reduced[:len(X_train_scaled)]
            X_test_reduced = X_reduced[len(X_train_scaled):]
        else:
            X_train_reduced = reducer.fit_transform(X_train_scaled)
            X_test_reduced = reducer.transform(X_test_scaled)
        
        # Train the model
        model.fit(X_train_reduced, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test_reduced)
        
        # Calculate MSE
        mse = mean_squared_error(y_test, y_pred)
        
        # Store results
        results[name][model.__class__.__name__] = mse
        predictions[name][model.__class__.__name__] = y_pred

# Print results
for name, model_results in results.items():
    print(f"Results using {name}:")
    for model, mse in model_results.items():
        print(f"MSE using {model}: {mse}")

# Show 5 predictions from each reduction algorithm along with the expected value
for name, preds in predictions.items():
    print(f"Predictions using {name}:")
    for model, y_pred in preds.items():
        print(f"Model: {model}")
        for i in range(5):
            print(f"Example {i+1}: Predicted: {y_pred[i]}, Expected: {y_test.values[i]}")


C:\Users\marcb\anaconda3\envs\greystone\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\marcb\anaconda3\envs\greystone\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\marcb\anaconda3\envs\greystone\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\marcb\anaconda3\envs\greystone\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\mar

Results using PCA:
MSE using Ridge: 425.0980652312025
MSE using DecisionTreeRegressor: 2.8611380851278
MSE using GradientBoostingRegressor: 3.0103578699223
MSE using RandomForestRegressor: 1.9137723882078919
MSE using AdaBoostRegressor: 2.249647977005227
MSE using KNeighborsRegressor: 1.6031238633947644
MSE using MLPRegressor: 78454.3997971499
MSE using ElasticNet: 1831.2664165375397
MSE using SGDRegressor: 604.9608276636142
MSE using SVR: 1.2385127307988635
MSE using BayesianRidge: 475.5162594788319
MSE using KernelRidge: 426.71505574017647
MSE using LinearRegression: 423.7555920573327
MSE using RANSACRegressor: 1076.179706680776
MSE using TheilSenRegressor: 464.7494067348092
Results using t-SNE:
MSE using Ridge: 3.099886139324699
MSE using DecisionTreeRegressor: 4.448630050732383
MSE using GradientBoostingRegressor: 3.7022145809156153
MSE using RandomForestRegressor: 1.7034690242169277
MSE using AdaBoostRegressor: 2.7601242861804143
MSE using KNeighborsRegressor: 1.8140521664513323
M

In [5]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA, FastICA, KernelPCA, TruncatedSVD
from sklearn.manifold import TSNE, Isomap, LocallyLinearEmbedding
from sklearn.random_projection import GaussianRandomProjection
from sklearn.metrics import mean_squared_error
from umap import UMAP
from sklearn.linear_model import Ridge, ElasticNet, SGDRegressor, BayesianRidge, LinearRegression, RANSACRegressor, TheilSenRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.kernel_ridge import KernelRidge

# Load the data
file_path = "data_all_numerical_select_reduced.xlsx"
data = pd.read_excel(file_path)

# Define columns
data_columns = [
    'OF2', 'OF3', 'OF4', 'OF5', 'OF6', 'OF7', 'OF8', 'OF9', 'OF10', 'OF11', 'OF12', 'OF13', 'OF14', 'OF15', 'OF16', 'OF17', 'OF18', 'OF19', 'OF20',
    'OF21', 'OF22', 'OF23', 'OF24', 'OF25', 'OF26', 'OF27', 'OF28', 'OF29', 'OF30', 'OF31', 'OF32', 'OF33', 'OF34', 'OF37', 'OF38', 'F1', 'F2',
    'F3_1', 'F3_2', 'F3_3', 'F3_4', 'F3_5', 'F3_6', 'F3_7', 'F4', 'F5', 'F6', 'F7', 'F8', 'F9', 'F10', 'F11', 'F12', 'F13', 'F14', 'F15', 'F16',
    'F17', 'F18', 'F19', 'F20', 'F21', 'F22', 'F23', 'F24', 'F25', 'F26', 'F27', 'F28', 'F29', 'F30', 'F31', 'F32', 'F33', 'F34', 'F35', 'F36',
    'F37', 'F38', 'F39', 'F40', 'F41', 'F42', 'F43', 'F44', 'F45', 'F46', 'F47', 'F48', 'F49', 'F50', 'F51', 'F52', 'F53', 'F54', 'F55', 'F56',
    'F57', 'F58', 'F59', 'F62', 'F63', 'F64', 'F65', 'F66', 'F67', 'F68', 'S1', 'S2', 'S3', 'S4', 'S5', 'S6', 'PC', 'FC', 'WRI', 'SVT', 'VCHWC',
    'HWCC', 'MC', 'PP', 'ST', 'SWP', 'DP', 'ADLM', 'ATDO', 'AOD'
]

results_columns = ['SR']

# Prepare data for regression
X = data[data_columns]
y = data[results_columns[0]]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Implement dimensionality reduction techniques
dimensionality_reduction_techniques = {
    "PCA": PCA(n_components=10),
    "t-SNE": TSNE(n_components=2),
    "UMAP": UMAP(n_components=10),
    "Isomap": Isomap(n_components=10),
    "LLE": LocallyLinearEmbedding(n_components=10),
    "Autoencoders": TruncatedSVD(n_components=10),  # Assuming TruncatedSVD as a simple autoencoder
    "ICA": FastICA(n_components=10),
    "Kernel PCA": KernelPCA(n_components=10),
    "Random Projection": GaussianRandomProjection(n_components=10)  # Assuming GaussianRandomProjection
}

# Define the models
models = [
    Ridge(), DecisionTreeRegressor(), GradientBoostingRegressor(),
    RandomForestRegressor(), AdaBoostRegressor(), KNeighborsRegressor(),
    MLPRegressor(max_iter=200), ElasticNet(max_iter=1000), SGDRegressor(max_iter=1000),
    SVR(cache_size=1000), BayesianRidge(max_iter=1000), KernelRidge(),
    LinearRegression(), RANSACRegressor(), TheilSenRegressor()
]

# Train and evaluate each model with each dimensionality reduction technique
results = {}
predictions = {}
for name, reducer in dimensionality_reduction_techniques.items():
    results[name] = {}
    predictions[name] = {}
    for model in models:
        # Transform the data
        if name == "LDA":
            X_train_reduced = reducer.fit_transform(X_train_scaled, y_train)
            X_test_reduced = reducer.transform(X_test_scaled)
        elif name == "t-SNE":
            X_concatenated = np.concatenate((X_train_scaled, X_test_scaled), axis=0)
            X_reduced = reducer.fit_transform(X_concatenated)
            X_train_reduced = X_reduced[:len(X_train_scaled)]
            X_test_reduced = X_reduced[len(X_train_scaled):]
        else:
            X_train_reduced = reducer.fit_transform(X_train_scaled)
            X_test_reduced = reducer.transform(X_test_scaled)
        
        # Train the model
        model.fit(X_train_reduced, y_train)
        
        # Make predictions
        y_pred = model.predict(X_test_reduced)
        
        # Calculate MSE
        mse = mean_squared_error(y_test, y_pred)
        
        # Store results
        results[name][model.__class__.__name__] = mse
        predictions[name][model.__class__.__name__] = y_pred

# Print results
for name, model_results in results.items():
    print(f"Results using {name}:")
    for model, mse in model_results.items():
        print(f"MSE using {model}: {mse}")

# Show 5 predictions from each reduction algorithm along with the expected value
for name, preds in predictions.items():
    print(f"Predictions using {name}:")
    for model, y_pred in preds.items():
        print(f"Model: {model}")
        for i in range(5):
            print(f"Example {i+1}: Predicted: {y_pred[i]}, Expected: {y_test.values[i]}")


C:\Users\marcb\anaconda3\envs\greystone\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\marcb\anaconda3\envs\greystone\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\marcb\anaconda3\envs\greystone\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\marcb\anaconda3\envs\greystone\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\mar

Results using PCA:
MSE using Ridge: 5195.957423785418
MSE using DecisionTreeRegressor: 10.257559234162507
MSE using GradientBoostingRegressor: 4.555448597523405
MSE using RandomForestRegressor: 3.69867771499331
MSE using AdaBoostRegressor: 3.505702711155886
MSE using KNeighborsRegressor: 4.11799236210327
MSE using MLPRegressor: 70227.58262376025
MSE using ElasticNet: 2682.4608279647273
MSE using SGDRegressor: 5557.353860837162
MSE using SVR: 3.1261856987667964
MSE using BayesianRidge: 5303.183503974189
MSE using KernelRidge: 5113.757484230677
MSE using LinearRegression: 5192.754832105871
MSE using RANSACRegressor: 2537.889093941545
MSE using TheilSenRegressor: 5781.379327730146
Results using t-SNE:
MSE using Ridge: 4.483578084715555
MSE using DecisionTreeRegressor: 8.200248282847113
MSE using GradientBoostingRegressor: 3.455679604782507
MSE using RandomForestRegressor: 3.014227589565993
MSE using AdaBoostRegressor: 5.423698871635058
MSE using KNeighborsRegressor: 4.759617314151596
MSE 